In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH tab02a AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_biweekly
    WHERE calday = (SELECT MAX(calday) FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_biweekly)
),
asrt_list AS (
    SELECT DISTINCT 
        a.store_id, 
        CASE
            WHEN b.business_unit = '1500' THEN 'Supermarket'
            WHEN b.business_unit = '2000' AND b.concept IN ('WiN','Rural') THEN b.concept
            ELSE 'Urban'
        END business_model,
        a.product_id
    FROM tab02a a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
    WHERE b.business_unit IN ('1500', '2000', '6000') 
        AND b.store_status_desc = "Đang hoạt động"
        AND UPPER(b.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(b.store_name) NOT LIKE 'OUTLET%' 
        AND UPPER(b.store_name) NOT LIKE '%HUB%' 
        AND UPPER(b.store_name) NOT LIKE '%ONLINE%'
        AND UPPER(b.store_name) NOT LIKE '%KHO %'
        AND UPPER(b.store_name) NOT LIKE '%B2B%'
        AND a.store_id NOT LIKE 'GH%'
),
d_region_dc AS (
    SELECT 
        dc_id,
        region
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_region_dc
),
dc_sku AS (
    SELECT DISTINCT
        b.dc_id,
        a.product_id,
        a.is_out
    FROM (
        SELECT *
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku_biweekly
        WHERE calday = (SELECT MAX(calday) FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku_biweekly)
    ) a
    INNER JOIN d_region_dc b ON a.region = b.region
    
),
bu_city_mnf_bz_model_asm_dc_re_model AS (
    SELECT DISTINCT 
        a.business_unit,
        a.city,
        a.manufacturer_id,
        IF(c.SUB_MANUFACTURER_ID IS NOT NULL, c.SUB_MANUFACTURER_ID, b.SUB_MANUFACTURER_ID) SUB_MANUFACTURER_ID,
        IF(c.SUB_MANUFACTURER_ID IS NOT NULL, c.DC_ID, a.DC_ID) DC_ID,
        IF(c.SUB_MANUFACTURER_ID IS NOT NULL, c.REPLENISHMENT_MODE, a.REPLENISHMENT_MODE) REPLENISHMENT_MODE,
        IF(c.SUB_MANUFACTURER_ID IS NOT NULL, c.XD_DC_ID, a.XD_DC_ID) XD_DC_ID
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_bu_city_mnf_dc_re_mode a
    LEFT JOIN(SELECT DISTINCT manufacturer_id,sub_manufacturer_id from {catalog_name}.udp_wcm_gold_vcm_dwh.d_product ) b
    using (manufacturer_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_bu_city_mnf_submnf_dc_re_mode c 
    on a.business_unit = c.business_unit AND a.city = c.city AND b.manufacturer_id = c.manufacturer_id AND b.sub_manufacturer_id = c.sub_manufacturer_id 
),
dc_sku_active AS (
    SELECT DISTINCT 
        a.store_id dc_id,
        a.product_id
    FROM tab02a a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
    WHERE b.business_unit = '1100'
),
combine AS (
    SELECT
        a1.dc_id,
        a1.xd_dc_id,
        b.business_unit,
        a.store_id,
        b.store_name,
        b.is_win_plus,
        a.business_model,
        b.region_domain_vn mien,
        b.city,
        c.mch2_id,
        a.product_id,
        c.product_name,
        c.manufacturer_id,
        c.manufacturer_name,
        d.core_range assortment_type,
        a3.is_out,
        a1.replenishment_mode,
        IF(a2.dc_id IS NULL, 'INACTIVE', 'ACTIVE') dc_sku_status
    FROM asrt_list a 
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment d ON
            a.store_id = d.store_id AND
            a.product_id = d.product_id
        LEFT JOIN bu_city_mnf_bz_model_asm_dc_re_model a1 ON
            b.business_unit = a1.business_unit AND
            b.city = a1.city AND
            c.manufacturer_id = a1.manufacturer_id AND
            c.sub_manufacturer_id = a1.sub_manufacturer_id
        LEFT JOIN dc_sku_active a2 ON a2.dc_id = a1.dc_id AND a2.product_id = a.product_id
        LEFT JOIN dc_sku a3 ON a1.dc_id = a3.dc_id AND a.product_id = a3.product_id
    WHERE a1.dc_id IS NOT NULL AND a3.product_id IS NOT NULL
        AND c.mch2_id IN ('101','102','201','202','203')
),
main AS (
    SELECT *
    FROM combine
    WHERE mch2_id <> '101'
    UNION ALL
    SELECT a.*
    FROM (SELECT * FROM combine WHERE mch2_id = '101') a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_dc_fresh_sku b
        ON a.dc_id = b.dc_id AND a.product_id = b.product_id
),
win_plus AS (
    SELECT
        IF(a.dc_id = '1312' AND a.is_win_plus = 'YES', '1340', a.dc_id) dc_id,
        a.* EXCEPT(dc_id)
    FROM main a
),
buildstock_golive AS (
    SELECT
        IF(b.dc_bs_id IS NULL, a.dc_id, b.dc_bs_id) dc_id,
        IF(b.dc_bs_id IS NULL, a.replenishment_mode, "BS") replenishment_mode,
        a.* EXCEPT(dc_id, replenishment_mode)
    FROM win_plus a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive b USING (dc_id, product_id, business_unit)
),
flowthrough_golive AS (
    SELECT
        IF(b.dc_ft_id IS NULL, a.dc_id, b.dc_ft_id) dc_id,
        IF(b.dc_ft_id IS NULL, a.replenishment_mode, "FT") replenishment_mode,
        a.* EXCEPT(dc_id, replenishment_mode)
    FROM buildstock_golive a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive b USING (dc_id, product_id, business_unit)
)

SELECT 
    a.dc_id,
    b.store_name dc_name,
    a.* EXCEPT(dc_id)
 FROM flowthrough_golive a
 LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.dc_id = b.store_id;
 """)
 

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly_his
(
calday TIMESTAMP,
dc_id string,
dc_name string,
replenishment_mode string,
product_id string,
business_unit string,
store_id string,
store_name string,
is_win_plus string,
business_model string,
mien string,
city string,
mch2_id string,
product_name string,
manufacturer_id string,
manufacturer_name string,
assortment_type string,
is_out string,
dc_sku_status string
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly_his WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    dc_id,
    dc_name,
    replenishment_mode,
    product_id,
    business_unit,
    store_id,
    store_name,
    is_win_plus,
    business_model,
    mien,
    city,
    mch2_id,
    product_name,
    manufacturer_id,
    manufacturer_name,
    assortment_type,
    is_out,
    dc_sku_status
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_biweekly
""")